In [1]:
# libraries
from pathlib import Path
import re
from markdown import markdown
from bs4 import BeautifulSoup


# directories
root = Path(".")
input = root / "input.md"
index = Path("index.html")
math_page = root / "pages/math.html"
code_page = root / "pages/code.html"
CV_page = root / "pages/CV.html"
math_cards_archive = root / "resources/math_cards_archive.md"
code_cards_archive = root / "resources/code_cards_archive.md"
skeleton = root / "templates/skeleton.html"
README = Path("README.md")

# elements
card = """
<div class="w3-white w3-text-dark-grey w3-card-4 w3-margin-bottom card">
    <div class="w3-container {classes}">
        {content}
    </div>
</div>
"""

button = """
<button class="w3-button w3-block w3-grey {classes}">
    {content}
</button>
"""


# patterns
sensitive_patterns = {
    "BLOCK-MATH": r"\\\[.*?\\\]",
    "INLINE-MATH": r"\\\(.*?\\\)",
}


patterns_substitutes = [
    (
        r"<!-- begin solution -->\s*(.*?)\s*<!-- end solution -->",
        r'<div class="solution w3-margin-bottom"><b class="w3-text-green">Solution. </b>\1 &#x25A2;</div>',
    ),
    (
        r"<!-- figure:\s*(.*?)\s*-->\s*(<img.*?>)",
        r'<figure class="w3-text-grey w3-center">\2<figcaption>\1</figcaption></figure>',
    ),
    (
        r"```python\s*(.*?)\s*```",
        r'<pre><code class="language-py">\1</code></pre>',
    ),
    (
        r"(BLOCK-MATH)",
        r'<div class="block-math">\1</div>',
    ),
]


# functions
def soupify(file_name):
    with open(file_name, "r") as file:
        content = file.read()
    return BeautifulSoup(content, "lxml")


def stringify(file_name):
    with open(file_name, "r") as file:
        string = file.read()
    return string


def format_markdown(string):
    string = string.strip()
    patterns_substitutes = [
        (
            r"\$\$(.*?)\$\$",
            r"\[\1\]",
        ),
        (
            r"(?<![\$\\])\$(?![\$])(.*?)(?<![\$\\])\$(?![\$])",
            r"\(\1\)",
        ),
    ]

    for pattern, substitute in patterns_substitutes:
        string = re.sub(pattern, substitute, string, flags=re.DOTALL)

    return string


def quarantine(string):
    sensitives = dict()
    for id, pattern in sensitive_patterns.items():
        sensitives[id] = re.findall(pattern, string, flags=re.DOTALL)
        string = re.sub(
            pattern,
            id,
            string,
            flags=re.DOTALL,
        )

    return string, sensitives


def unquarantine(string, sensitives):
    for id, substitutes in sensitives.items():
        if id == "BLOCK-MATH" or "INLINE-MATH":
            substitutes = [re.sub(r"%", "", substitute) for substitute in substitutes]

        string = re.sub(id, lambda _: substitutes.pop(0), string, flags=re.DOTALL)
    return string


def parse(string):
    string, sensitives = quarantine(string)
    for pattern, substitute in patterns_substitutes:
        string = re.sub(pattern, substitute, string, flags=re.DOTALL)

    html = markdown(string)

    html = unquarantine(html, sensitives)
    return html


def append_cards(
    content: str | Path,
    page: BeautifulSoup | Path,
    classes: str = "",
    archive: Path = None,
) -> BeautifulSoup:
    content = stringify(content) if isinstance(content, Path) else content
    strings = (
        content.split(r"<!-- separator -->")[:-1]
        if r"<!-- separator -->" in content
        else [content]
    )
    page_soup = soupify(page) if isinstance(page, Path) else page

    for string in strings:
        string = format_markdown(string)
        html = parse(string)

        card_soup = BeautifulSoup(
            card.format(content=html, classes=classes), "lxml"
        ).div
        if card_soup.select_one(".solution"):
            button_soup = BeautifulSoup(
                button.format(content="Show/Hide Solution", classes="toggle-solution"),
                "lxml",
            ).button
            card_soup.append(button_soup)

        page_soup.select_one("#cards-container").append(card_soup)

    if not archive is None:
        with open(archive, "a") as file:
            for string in strings:
                file.write(string.strip() + "\n\n<!-- separator -->\n\n")

    return page_soup


def write(page, soup):
    with open(page, "w") as file:
        file.write(str(soup))

In [3]:
write(math_page, append_cards(math_cards_archive, skeleton))